# Feature Engineering

In [1]:
import pandas as pd
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

from tqdm import tqdm

import time
import torch
import gc

pd.set_option('display.max_columns', None)

## Daten laden

In [2]:
IS_TRAINING = False
FILE = '../../data/processed/validation_series_split_normalized.parquet'
series = pd.read_parquet(FILE, columns=['num_series_id'])

## Neue Features erstellen

In [3]:
folder = "train/" if IS_TRAINING else "validation/"
file_path_prefix = "../../data/processed/lag-features/" + folder

ANGLEZ_LAGS_FUTURE = [f"anglez_future-lag_{i}" for i in range(-1, -25, -1)]
ENMO_LAGS_FUTURE = [f"enmo_future-lag_{i}" for i in range(-1, -25, -1)]
ANGLEZ_LAGS_PAST = [f"anglez_past-lag_{i}" for i in range(1, 25)]
ENMO_LAGS_PAST = [f"enmo_past-lag_{i}" for i in range(1, 25)]
LAGS_FUTURE = [*ANGLEZ_LAGS_FUTURE, *ENMO_LAGS_FUTURE]
LAGS_PAST = [*ANGLEZ_LAGS_PAST, *ENMO_LAGS_PAST]
FEATURES = ['anglez', 'enmo', *LAGS_PAST, *LAGS_FUTURE]

LABEL = ['awake']

In [4]:
def save_chunk(num_series_id, chunk):
    X = torch.from_numpy(chunk[FEATURES].astype('float32').to_numpy())
    
    torch.save(X, file_path_prefix + str(num_series_id) + ".pt")

In [5]:
def make_features_chunk(num_series_id):
    df = pd.read_parquet(FILE, filters=[('num_series_id','=',num_series_id)])
          
    for i in range(1, 25):        
        df[f"anglez_past-lag_{i}"] = df["anglez"].shift(i).bfill()
        df[f"enmo_past-lag_{i}"] = df["enmo"].shift(i).bfill()

    for i in range(-1, -25, -1):
        df[f"anglez_future-lag_{i}"] = df["anglez"].shift(i).ffill()
        df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
    
    return df.reset_index(drop=True)

In [6]:
def make_features(series):
    overview_data = []
    
    for num_series_id in tqdm(series.num_series_id.unique()):
        chunk = make_features_chunk(num_series_id)
        save_chunk(num_series_id, chunk)
        
        overview_data.append(
            chunk[['num_series_id', 'awake']].reset_index().rename(columns={'index':'series_index'}).copy()[['num_series_id', 'awake', 'series_index']]
        )
        
        del chunk
        gc.collect()
    
    return pd.concat(overview_data).reset_index(drop=True)

In [7]:
start_time = time.time()
overview = make_features(series)
print(f'Feature Engineering took {time.time() - start_time} seconds')

  0%|                                                                                           | 0/54 [00:00<?, ?it/s]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
  2%|█▌                                                                                 | 1/54 [00:03<03:16,  3.72s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 30%|████████████████████████▎                                                         | 16/54 [00:41<01:38,  2.59s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 31%|█████████████████████████▊                                                        | 17/54 [00:43<01:32,  2.50s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 59%|████████████████████████████████████████████████▌                                 | 32/54 [01:25<01:04,  2.92s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 61%|██████████████████████████████████████████████████                                | 33/54 [01:26<00:53,  2.55s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

 89%|████████████████████████████████████████████████████████████████████████▉         | 48/54 [02:05<00:15,  2.51s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_future-lag_{i}"] = df["enmo"].shift(i).ffill()
 91%|██████████████████████████████████████████████████████████████████████████▍       | 49/54 [02:11<00:17,  3.41s/it]C:\Users\timon\AppData\Local\Temp\ipykernel_31544\2435301155.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"enmo_f

Feature Engineering took 144.1399416923523 seconds


## Overview speichern

In [8]:
overview

,num_series_id,awake,series_index
0,7,1,0
1,7,1,1
2,7,1,2
3,7,1,3
4,7,1,4
...,...,...,...
20009335,276,1,620635
20009336,276,1,620636
20009337,276,1,620637
20009338,276,1,620638


In [9]:
overview.to_parquet(file_path_prefix + "overview.parquet")